In [ ]:
# === 1️⃣ Install required packages ===
!pip install streamlit
!pip install pyngrok
!pip install youtube-transcript-api
!pip install transformers
!pip install torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.1/485.1 kB 9.6 MB/s eta 0:00:00


In [ ]:
# === 2️⃣ Write Streamlit app to app.py ===
%%writefile app.py
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import pipeline
import streamlit as st

# Helper function to extract video ID
def extract_video_id(url: str) -> str:
    parsed = urlparse(url)
    qs = parse_qs(parsed.query)
    video_ids = qs.get('v')
    if not video_ids:
        raise ValueError(f"No video id found in URL: {url}")
    return video_ids[0]

# Streamlit App
st.title("🎬 YouTube Video Summarizer")
st.write("Enter a YouTube video URL to get a summarized transcript.")

url = st.text_input("YouTube URL")

if st.button("Summarize"):
    if url:
        try:
            video_id = extract_video_id(url)

            # ✅ Correct way to fetch transcript with new API
            api = YouTubeTranscriptApi()
            fetched = api.fetch(video_id, languages=["en"])
            text = " ".join([snippet["text"] for snippet in fetched.to_raw_data()])

            # Load summarizer
            summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

            # Chunking
            words = text.split()
            chunk_size = 500
            summaries = []

            with st.spinner("Summarizing video... This may take a few seconds."):
                for i in range(0, len(words), chunk_size):
                    chunk_text = " ".join(words[i:i+chunk_size])
                    summary = summarizer(chunk_text, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
                    summaries.append(summary)

            final_summary = " ".join(summaries)
            st.subheader("📝 Summary")
            st.write(final_summary)

        except Exception as e:
            st.error(f"Error: {str(e)}")
    else:
        st.warning("Please enter a valid YouTube URL.")


Overwriting app.py


In [ ]:
from pyngrok import ngrok

# Connect directly to port 8501
public_url = ngrok.connect(8501, "http")
print(f"🌐 Your Streamlit app is live at: {public_url}")


🌐 Your Streamlit app is live at: NgrokTunnel: "https://46d14498b539.ngrok-free.app" -> "http://localhost:8501"


In [12]:
# Run the app
!streamlit run app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.243.234.114:8501

2025-11-17 23:45:22.664104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763423122.691875    3514 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763423122.701795    3514 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763423122.722642    3514 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1763423122.722684    3514 computation_placer.cc:177] computation placer alr